In [262]:
from abc import abstractmethod
# from pykrx.website.comm.webio import Get, Post
import requests
import pandas as pd
from pandas import DataFrame
from datetime import datetime, timedelta
import time

In [322]:
class Post:
    def __init__(self) -> None:
        self.headers = {"User-Agent": "Mozilla/5.0"}

    def read(self, **params):
        # print(params)
        resp = requests.post(self.url, headers=self.headers, data=params)
        return resp.json()

    @property
    @abstractmethod
    def url(self):
        return NotImplementedError

In [315]:
class WebAPI(Post):    
    # read with period
    def read_period(self, start, end, **params):
        time.sleep(0.5)
        delta = timedelta(days=365)
        day = timedelta(days=1)
        if start > end: return
        elif start + delta < end:
            curr_end = start + delta
            params['strtDd'] = start.strftime("%Y%m%d")
            params['endDd'] = curr_end.strftime("%Y%m%d")
            result = super().read(**params)
            result['output'] = self.read_period(curr_end + day, end, **params)['output'] \
                                + result['output']
            return result
        else:
            params['strtDd'] = start.strftime("%Y%m%d")
            params['endDd'] = end.strftime("%Y%m%d")
            result = super().read(**params)
            return result
            
    def read(self, **params):
        result = None
        params.update(bld=self.bld)

        if not 'strtDd' in params or not 'endDd' in params:
            return super().read(**params)

        start_date = pd.to_datetime(params['strtDd'])
        end_date = pd.to_datetime(params['endDd'])
        return self.read_period(start_date, end_date, **params)
        
    @property
    @abstractmethod
    def bld(self):
        return NotImplementedError

    @property
    def url(self):
        return "http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd"

In [316]:
class 개별종목시세추이(WebAPI):
    def load(self, stock_code: str, start: str, end: str):
        result = self.read(isuCd=stock_code,
                            strtDd=start,
                            endDd=end)
        return DataFrame(result['output'])
    @property
    def bld(self):
        return "dbms/MDC/STAT/standard/MDCSTAT01701"

In [317]:
class 주식종목검색(WebAPI):
    def search(self, searchText: str):
        result = self.read(searchText=searchText,
                            mktsel='ALL',
                            typeNo='0')
        return DataFrame(result['block1'])
    @property
    def bld(self):
        return "dbms/comm/finder/finder_stkisu"

In [318]:
class KrxStock:
    def get_stock_code(self, stock_name: str):
        return 주식종목검색().search(stock_name)
    def get_stock_OHLCV(self, stock_code: str, start: str, end: str):
        return 개별종목시세추이().load(stock_code, start, end)

In [269]:
class 개별지수종합정보(WebAPI):
    def load(self, index_id: str, index_group_id, date: str) -> DataFrame:
        result = self.read(indIdx2=index_id,
                            indIdx=index_group_id,
                            trdDd=date)
        return DataFrame(result['output'])
    @property
    def bld(self):
        return "dbms/MDC/STAT/standard/MDCSTAT00601"

In [329]:
class KrxIndex:
    def __init__(self):
        self.index_list = {
            "KOSPI" : {"index_id": "001", "index_group_id": "1"}, 
            "KOSPI 200" : {"index_id": "028", "index_group_id": "1"}, 
            "KOSDAQ" : {"index_id": "001", "index_group_id": "2"}, 
            "KOSDAQ 150" : {"index_id": "203", "index_group_id": "2"}
        }

    def unpack_index_id(self, index_id: str, index_group_id: str):
            return index_id, index_group_id

    def get_index_constituents(self, index_name: str, date: str):
        try:
            index_id, index_group_id = self.unpack_index_id(**self.index_list[index_name])
        except:
            print(f"Select index in {[key for key in self.index_list.keys()]}")
            return None
        
        data = 개별지수종합정보().load(index_id, index_group_id, date)
        return data

    def get_index_constituents_OHLCV(self, index_name: str, start: str, end: str):
        start_date = pd.to_datetime(start)
        end_date = pd.to_datetime(end)
        delta = pd.to_timedelta('1 days')
        result = pd.DataFrame()
        result['stock_name'] = ''
        while start_date <= end_date:
            data = self.get_index_constituents(index_name, start)
            for stock_name in data['ISU_ABBRV']:
                full_code = KrxStock().get_stock_code(stock_name)['full_code'].iloc[0]
                curr = start_date.strftime("%Y%m%d")
                result.at[-1,'stock_name'] = stock_name
                result.append(KrxStock().get_stock_OHLCV(full_code, curr, curr))
            start_date += delta

        return result

In [335]:
class WebSource():
    def __init__(self):
        self.krxStock = KrxStock()
        self.krxIndex = KrxIndex()

In [338]:
ws = WebSource()
print(ws.krxStock.get_stock_code('삼성전자'))
print(ws.krxStock.get_stock_OHLCV(code, '20230818', '20230818'))
print(ws.krxIndex.get_index_constituents('KOSPI', '20230818'))

{'searchText': '삼성전자', 'mktsel': 'ALL', 'typeNo': '0', 'bld': 'dbms/comm/finder/finder_stkisu'}
      full_code short_code codeName marketCode marketName marketEngName ord1  \
0  KR7005930003     005930     삼성전자        STK       유가증권         KOSPI    1   
1  KR7005931001     005935    삼성전자우        STK       유가증권         KOSPI    1   

  ord2  
0   16  
1   16  
{'isuCd': 'KR7005930003', 'strtDd': '20230818', 'endDd': '20230818', 'bld': 'dbms/MDC/STAT/standard/MDCSTAT01701'}
       TRD_DD TDD_CLSPRC FLUC_TP_CD CMPPREVDD_PRC FLUC_RT TDD_OPNPRC  \
0  2023/08/18     66,300          2          -400   -0.60     66,000   

  TDD_HGPRC TDD_LWPRC  ACC_TRDVOL       ACC_TRDVAL               MKTCAP  \
0    66,700    65,800  11,745,006  778,550,834,100  395,796,583,065,000   

       LIST_SHRS  
0  5,969,782,550  
{'indIdx2': '001', 'indIdx': '1', 'trdDd': '20230818', 'bld': 'dbms/MDC/STAT/standard/MDCSTAT00601'}
    ISU_SRT_CD ISU_ABBRV TDD_CLSPRC FLUC_TP_CD STR_CMP_PRC FLUC_RT  \
0       005930  